<a href="https://colab.research.google.com/github/LivingstonTardzenyuy/Deep-Learning-with-TensorFlow/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences text or speech)

Another common term for NLP problems is sequence to sequence problems(seq2seq).

## Check for GPU

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-26e0b78b-2b56-1e22-4720-bb5d20634ebc)


## Get helper functions.

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2025-01-09 01:46:02--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-01-09 01:46:03 (63.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get our text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as diaster or not diaster).

See the original source here: https://www.kaggle.com/competitions/nlp-getting-started

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip the data.
unzip_data("nlp_getting_started.zip")

--2025-01-09 01:46:14--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.107.207, 142.251.188.207, 108.177.98.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.107.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2025-01-09 01:46:14 (118 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do that is using Python.

But it's good we get visual straight away. Hence we're going to use pandas.

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

# Shuffle training dataFrame.

The reason we shuffle our data is to make sure our model don't have a particular format to learn(so it won't lead to baises).

In [ ]:
# Shuffle our training dataset.
train_df_shuffled = train_df.sample(frac=1, random_state=42)   # frac=1 means shuffle all the data.
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
# What does the test dataFrame look like ?.
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
## How many examples of each class are they ?.

train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [ ]:
# How many total samples ?.
len(train_df), len(test_df)

(7613, 3263)

In [ ]:
# Let's visualize some random training examples.
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index: random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
New Explosion-proof Tempered Glass Screen Protector Film for Blackberry Z10 - Full read byÛ_ http://t.co/z4IlB9y9nU http://t.co/j295MD1SOW

---

Target: 1 (real disaster)
Text:
Small casualty on the way to Colorado http://t.co/hDVmhSQXHm

---

Target: 0 (not real disaster)
Text:
Wired: Reddit Will Now Quarantine Offensive Content - Reddit co-founder and CEO Steve Huffman has unveiled more sp... http://t.co/aByHRgsS1s

---

Target: 0 (not real disaster)
Text:
Heather Night and Ava Sparxxx enjoy a wild teen threesome http://t.co/qew4c5M1xd View and download video

---

Target: 0 (not real disaster)
Text:
#BakeOffFriends the one with the mudslide

---



### Split data into training and Validation datasets.

In [ ]:
from sklearn.model_selection import train_test_split

train_sentence, val_sentence, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(),
    train_df_shuffled["target"].to_numpy(),
    test_size=0.1,   # Use 10% for validation data.
    random_state=42
)

# Check length of our sentences.
len(train_sentence), len(train_labels), len(val_sentence), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
# Check out the first 10 samples.
train_sentence[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with text problems, one of the first things you'll have to do before you build a model is to convert your text to numbers.

There are few ways to do this, namely:
* Tokenization: directly mapping a token(a token could be a word or character) to number.
* Embedding - create a matrix of feature vector for each token



```
# This is formatted as code
```

### Text Vectorization(tokenization)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
# Use the default text vectorization parameters.
text_vectorizer = TextVectorization(
    max_tokens = 10000,   # how many words in the vocabulary. None means we don't know how many words we can have.
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,   # create groups of n-words?
    output_mode='int',   # how to map tokens to numbers
    output_sequence_length=None,   # how long do you want your sequences to be ?.
    pad_to_max_tokens=True
)

text_vectorizer

<TextVectorization name=text_vectorization, built=False>

In [ ]:
train_sentence[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [ ]:
# Find the average number of tokens(words) in the training tweets
round(sum([len(i.split()) for i in train_sentence]) / len(train_sentence))

15

In [ ]:
# Setup text vectorization veriables.
max_vocab_length = 1000  # max number of words to have in our vocabulary.
max_length = 15 # max length our  sequences will be(Calculated from above).

text_vectorizer = TextVectorization(
    max_tokens=max_vocab_length,
    output_mode="int",
    output_sequence_length=max_length
)

In [ ]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentence)

In [ ]:
# Create a sample sentence and tokenize it.
sample_tensence = "Great i'm doing just awersome!"
text_vectorizer([sample_tensence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[196,  32, 487,  29,   1,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
# Choose randome sentence from the training dataset and tokenize it.
random_sentence = random.choice(train_sentence)
print(f"Original text:\n {random_sentence}\
      \n\n Vectorized version:")
text_vectorizer([random_sentence])

Original text:
 Here we go again. EBOLA! Run for your lives! aaarrrgghhh!

http://t.co/G2UIMBeKgE      

 Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[140,  46, 112, 282,   1, 340,  10,  33, 394,   1,   1,   0,   0,
          0,   0]])>

In [23]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")


Number of words in vocab: 1000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['reported', 'r', 'pray', 'playlist', 'patience']


### Creating an Embedding using an Embedding Layer

To make our embedding, we're going to use TensorFlow's embedding Layer.
The parameters we care most about for our embedding layer:
   * 'input_dim' = The size of our vocabulary
   * 'output_dim' = The size of the output embedding vector, for e.g A value of 100 means each token get represented by token of 100 long.
   * 'input_length' = Length of sequences of the sequences passed to the embedding layer.
   

In [24]:
from tensorflow.keras import layers

embedding = layers.Embedding(
    input_dim = max_vocab_length,
    output_dim = 128,
    input_length = max_length
)

embedding

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<Embedding name=embedding, built=False>

In [31]:
# Get a random sentence from the training set.
random_sentence = random.choice(train_sentence)
print(f"Original text:\n {random_sentence}\
        \n\nEmbedded version:")

# Embed the random sentence(convert it to a embedding).Turns positive integers (indexes) into dense vectors of fixed size.
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 I want to be with you forever
Stay by my side
On this special night

Fear and Loathing in Las Vegas/Solitude X'mas        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02255582,  0.0259901 ,  0.01694116, ..., -0.03740091,
         -0.00735105,  0.01949378],
        [ 0.00412489, -0.00584438, -0.02464767, ..., -0.02366555,
         -0.01416775,  0.02741282],
        [ 0.0472679 ,  0.03268227,  0.01048347, ...,  0.02652517,
          0.00375062,  0.02310852],
        ...,
        [-0.00277942, -0.02209071,  0.0137451 , ...,  0.02697461,
          0.03351324, -0.00042889],
        [-0.0018643 ,  0.02754964, -0.04849609, ...,  0.00614063,
         -0.02287358, -0.01442665],
        [ 0.0186999 , -0.03355801, -0.04905776, ..., -0.04089778,
          0.03272844,  0.04627151]]], dtype=float32)>

In [32]:
# Checkout a single token embedding.
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.02255582,  0.0259901 ,  0.01694116, -0.01024364,  0.02120984,
        -0.03151736, -0.03253974, -0.01355288, -0.04806685,  0.04478966,
        -0.02359766, -0.03837729, -0.04869689,  0.04242169,  0.01868388,
         0.01732831, -0.02083503, -0.02775868, -0.03467612, -0.00875461,
        -0.01151458,  0.01802235, -0.03445597, -0.02822069,  0.03876157,
         0.0447141 , -0.04567411, -0.02719765,  0.01299636,  0.00869652,
        -0.00735261, -0.04086207,  0.02211714,  0.04971144,  0.03558833,
         0.01299037, -0.0227052 ,  0.00130111,  0.0365786 , -0.04243313,
        -0.01283253, -0.02828985,  0.02876902,  0.04223312,  0.00949718,
        -0.02101237,  0.03112826,  0.00459284,  0.0351924 , -0.03575096,
        -0.04351821, -0.03770291, -0.01481025,  0.02860421, -0.04850334,
        -0.00302838,  0.0065369 , -0.04776033,  0.00381624, -0.01010376,
        -0.03936304,  0.03548012,  0.04381205,  0.03009203, -0.01990626,
  

## Modelling a text dataset

Now we've got  way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

we'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline), this is from sklearn ML model selction.
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor (usig transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data.

How are we going to approach all of these ?.andUse the standard steps in modelling with tensorflow.
  * Create a model
  * Build a model
  * Fit a model
  * Evaluate our model

In [36]:
len(train_sentence)

6851

In [35]:
len(sample_embed)

1

### Model 0: Getting a baseline.

As with all ML model experiments. it's important to create a benchmark to build a model.
To create our baseline we'll use our SKlearn Naive Bayes model..

**NB**: It's common practice to use None deep learning as our baseline(b/c of thier speed) then letter use deep learning for fine-tuning.........

In [40]:
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


# Create tokenization and modelling pipeline.
model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

# Fit the model.
model_0.fit(train_sentence, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [42]:
# Evaluate our baseline trained model.
baseline_score = model_0.score(val_sentence, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [45]:
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [47]:
# Make predictions..
baseline_preds = model_0.predict(val_sentence)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])